<a href="https://colab.research.google.com/github/JueunL/IANNWTF-Group25/blob/Workflow/IANNWTF_HW9_jo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from numpy.random import randint

## Generator

In [88]:
# The number of samples to be used for creating the dataset
NUM_SAMPLES = 100 # @param
# Number of Context numbers
LEN_CONTEXT = 2 # @param
# Length of the Sequence where the context numbers are counted
LEN_SEQUENCE = 23 # @param



In [122]:
def generate_samples(n=NUM_SAMPLES, len_seq=LEN_SEQUENCE, len_con=LEN_CONTEXT):
  """ Generate samples for the "count occurences" excercise using a 
  generator.

  Args:
    n (int): Number of samples to generate
    len_seq (int): Length of the Sequence where the context numbers are counted
    len_con (int): Number of Context numbers

  Returns:
    (np.array): Array including the context numbers
    (np.array): Array including the sequence
    (np.array): Multi-Hot array signifying which context number
                has the highest count in the sequence
                (if there are context numbers that share the highest count
                each will get a 1)
  """
  num = 0
  while num < n:
    # Generate random sequence and context of specified lengths
    sequence = randint(0,10,len_seq)
    context = randint(0,10,len_con)

    # Count how many instances of each context word were in the sequence
    counts = [np.count_nonzero(sequence == c) for c in context]

    # Create a multi-hot encoded list where only the context number(s) with the
    # highest count are 1 (else 0)
    counts_max = np.max(counts)
    label = [int(c == counts_max) for c in counts]

    # Yield the result to turn the function into a generator which saves RAM
    yield context, sequence, np.array(label)
    #yield np.concatenate((context, sequence)), np.array(label)

    num += 1

In [123]:
# Generate a tensorflow dataset from the generator with a tensor for
# context, sequence and label each
data = tf.data.Dataset.from_generator(
    generate_samples,
    output_signature=(
        tf.TensorSpec(shape=(LEN_CONTEXT), dtype=tf.int32),
        tf.TensorSpec(shape=(LEN_SEQUENCE), dtype=tf.int32),
        tf.TensorSpec(shape=(LEN_CONTEXT), dtype=tf.int32)
    ))